### Note
由於茼蒿和豌豆屬於季節性蔬菜，會有大半年沒有買賣資料，所以本次專案以花椰菜（青梗）為資料來源，建立蔬菜價格預測模型



## 檔案說明
這個檔案的目標是從weekly_veg_dataset.csv資料集中，整理出建模所需的資料集

In [11]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib

# 設定字型為微軟正黑體
matplotlib.rcParams['font.family'] = 'BiauKaiHK'

# 讀入資料
data = pd.read_csv("../data/weekly_veg_dataset.csv")

# 製作花椰菜資料集
cauliflower_data = data[data['品項'] == '花椰菜（青梗）']

df = cauliflower_data

# 設定單一市場
# df = df[df['市場'] == '514 溪湖鎮']


In [12]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os

def analyze_market_corr(df, market_code, plot=True, save_dir="coefficient"):

    df_market = df[df['市場'] == market_code]
    best_corrs = []

    for i in range(13):
        suffix = f"_w-{i}"
        columns = ["平均價_元每公斤"] + [col for col in df_market.columns if col.endswith(suffix)]
        df_sub = df_market[columns].dropna()

        corr_matrix = df_sub.corr()
        if not corr_matrix.empty and "平均價_元每公斤" in corr_matrix:
            corrs = corr_matrix["平均價_元每公斤"].drop("平均價_元每公斤")

            if not corrs.empty:
                max_pos_var = corrs.idxmax()
                max_pos_corr = corrs[max_pos_var]

                max_neg_var = corrs.idxmin()
                max_neg_corr = corrs[max_neg_var]

                best_corrs.append((f"w-{i}", max_pos_var, max_pos_corr, max_neg_var, max_neg_corr))

    # 找出最大正相關的週數
    max_corr_entry = max(best_corrs, key=lambda x: x[2])
    max_i = int(max_corr_entry[0].split("-")[1])
    max_i = 3

    # 印出每週的結果
    print(f"\n市場：{market_code}")
    print(f"最大正相關發生在 w-{max_i}，變數為 {max_corr_entry[1]}，相關係數 = {max_corr_entry[2]:.4f}\n")
    # for week, pos_var, pos_corr, neg_var, neg_corr in best_corrs:
    #     print(f"{week}:")
    #     print(f"  最大正相關：{pos_var} (相關係數 = {pos_corr:.4f})")

    # 畫圖 & 儲存
    if plot:
        suffix = f"_w-{max_i}"
        columns = ["平均價_元每公斤"] + [col for col in df_market.columns if col.endswith(suffix)]
        df_sub = df_market[columns].dropna()
        corr_matrix = df_sub.corr()

        plt.figure(figsize=(12, 10))
        sns.heatmap(
            corr_matrix[['平均價_元每公斤']].sort_values(by='平均價_元每公斤', ascending=False),
            annot=True, cmap='coolwarm'
        )
        title = f"{market_code} - 平均價相關係數 (w-{max_i})"
        plt.title(title, fontsize=14)

        # 建立資料夾（如果還沒存在）
        os.makedirs(save_dir, exist_ok=True)
        # 儲存為 JPG
        filename = f"{save_dir}/{market_code.replace(' ', '_')}_w-{max_i}.jpg"
        plt.savefig(filename, format='jpg', dpi=300, bbox_inches='tight')
        plt.close()

        print(f"圖已儲存：{filename}")

    return max_i, best_corrs


In [14]:
markets = [
    #'400 台中市',
    #'420 豐原區',
    '512 永靖鄉',
    # '514 溪湖鎮',
    # '648 西螺鎮'
]

for market in markets:
    analyze_market_corr(df, market)



市場：512 永靖鄉
最大正相關發生在 w-3，變數為 平均露點溫度(℃)_w-4，相關係數 = 0.7343

圖已儲存：coefficient/512_永靖鄉_w-3.jpg


/Users/qiongyuechen/miniconda3/envs/ML/lib/python3.11/site-packages/seaborn/utils.py:61: UserWarning: Glyph 8722 (\N{MINUS SIGN}) missing from font(s) BiauKaiHK.
  fig.canvas.draw()
/var/folders/cr/v3jr1ngj2zd1_8qj8cr2nj3m0000gn/T/ipykernel_38429/652474521.py:60: UserWarning: Glyph 8722 (\N{MINUS SIGN}) missing from font(s) BiauKaiHK.
  plt.savefig(filename, format='jpg', dpi=300, bbox_inches='tight')
/var/folders/cr/v3jr1ngj2zd1_8qj8cr2nj3m0000gn/T/ipykernel_38429/652474521.py:60: UserWarning: Glyph 8722 (\N{MINUS SIGN}) missing from font(s) BiauKaiHK.
  plt.savefig(filename, format='jpg', dpi=300, bbox_inches='tight')


修改欄位名稱（因為R語言的讀取會有問題）

In [3]:
import pandas as pd
import re

df = cauliflower_data

df['week'] = df['week'].astype(str).str.zfill(2)

# 用正則表達式移除括號與其中的內容
df.columns = [re.sub(r'\([^)]*\)', '', col).strip() for col in df.columns]

# 儲存為新的檔案
df.to_csv("cauliflower_cleaned.csv", index=False)
print(f"處理完成，已儲存為：cauliflower_cleaned.csv")



NameError: name 'cauliflower_data' is not defined